In [ ]:
#run in venv called 3DED on Erebor

In [1]:
    import numbers
    import os
    import math
    from pathlib import Path
    import numpy as np
    from ase.io import read
    from ase.visualize import view
    from ase.build import surface
    from ase.atom import Atom
    from ase.atoms import Atoms
    from scipy import io as out

In [2]:
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta (degrees) which we convert into radians.
    """
    theta = math.radians(theta)
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

In [38]:
cif_cell = 'C:\\Users\\Sauron\\hl585\\multem_stable\\helen_code\\acetaminophen_1998_monoclinic.cif' 
#cif_cell = 'C:\\Users\\Sauron\\hl585\\multem_stable\\helen_code\\mp-81_Au.cif' 
filepath = Path(cif_cell)
name = filepath.stem
path = filepath.parent

cif_cell = read(cif_cell)
print(path, name)

C:\Users\Sauron\hl585\multem_stable\helen_code acetaminophen_1998_monoclinic


In [39]:
sample_x = cif_cell.cell[0,0]
sample_y = cif_cell.cell[1,1]
sample_z = cif_cell.cell[2,2]
print(cif_cell.cell)
lx = [sample_x]
ly = [sample_y]
dz = [sample_z/2] # Preferably some fraction of the unit cell, or just a small distance
#i'm just trying dz = 2, where dz = slice thickness
dz = [2]

header = [lx + ly + dz + 5*[0]]

occ = [1]
label = [0]
charge = [0]

Cell([[7.0939, 0.0, 0.0], [0.0, 9.2625, 0.0], [-1.55623145814634, 0.0, 11.552653056708477]])


In [40]:
cif_cell *= (10,10,10)
cif_cell = surface(cif_cell, indices=(1, 0, 0), layers=1, periodic=True)
print(cif_cell.cell)

Cell([92.625, 116.57000000000001, 70.30399375395407])


In [41]:
rotated_array = np.empty([len(cif_cell.positions),3])
counter=0
for xyz in cif_cell.positions:
    #Atoms.euler_rotate(xyz, phi=1, psi=0.0, theta =0.0, center = (0,0,0))
    axis=[4,4,1]
    theta = -10
    x=xyz[0]
    y=xyz[1]
    z=xyz[2]
    v = [x, y, z]
    matrix= rotation_matrix(axis, theta)
    rotated = np.dot(matrix, v)

    rotated_array[counter] = rotated
    counter = counter+1

#print(rotated_array)
cif_cell.positions = rotated_array

In [42]:
#now for the rotated array, 'cookie-cutter' out the coordinates such that the shape of the input is aligned with the beam axis
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

#how large the cell we want to input into MULTEM is
input_cell = 70
#define the cell in the python shapely library
polygon = Polygon([(0, 0, 0), (0, input_cell, 0), (0,0,input_cell), (input_cell,input_cell,0), (input_cell, input_cell, input_cell), (input_cell, 0, input_cell)])

new_cell_array = np.empty([len(cif_cell.positions),3])
#check if each point is in the desired cell.
for i in range(len(rotated_array)):
    shape = rotated_array[i]
    x= shape[0]
    y= shape[1]
    z= shape[2]
    point = Point(x, y, z)
    #print(polygon.contains(point))
    
    #if the point is in the cell, write it into a new array
    if polygon.contains(point)==True: 
        #print(x,y,z)
        new_cell_array[i]=rotated_array[i]

#print(new_cell_array)


#bug in this code: when we rotate further and further, eventually the bounding boxes become totally irrelevant-
#the whole specimen shifts out of these boxes

In [43]:
cif_cell.positions= new_cell_array

In [44]:
rms3d =0.085
if isinstance(rms3d, numbers.Number):
    rms3d_list = [rms3d for Z in cif_cell.numbers]
else:
    rms3d_list = [rms3d[Z] for Z in cif_cell.numbers]

data = [[n] + list(xyz) + [rms3d] + occ + label + charge for n, xyz, rms3d in zip(cif_cell.numbers, cif_cell.positions, rms3d_list)]
total_with_blanks = np.array(header + data)
print(len(total_with_blanks))
#now get rid of the null coordinates (i.e. the atoms which fell outside of the unit cell shape)
total= []

for i in range(len(total_with_blanks)):
    array = total_with_blanks[i]
    if array[1]!=0 and array[2]!=0 and array[3]!=0:
        total.append(array)
print(len(total))
#print(total)
print(path)
np.savetxt(path / (name + f"{theta}_rotated.txt"), total, fmt='%.8f', newline="\n", delimiter=" ")

80001
16680
C:\Users\Sauron\hl585\multem_stable\helen_code
